In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import itertools

from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix


KeyboardInterrupt: 

In [ ]:
device = torch.device("cpu")

In [ ]:
class BinarizeTransform:
    """
    A class to binarize the input MNIST data.
    """
    def __call__(self, img):
        # Values are between 0 and 1 so I have binarized with threshold of 0.5
        return (img>0.5).float()

# Transform to be applied on to the data immediately after loading from location.
transform = transforms.Compose([
    transforms.ToTensor(),
    BinarizeTransform()
])

# Load and transform MNIST data
mnist_data_train = datasets.MNIST(root="data/MNIST", train=True, download=True, transform=transform)
mnist_data_test = datasets.MNIST(root="data/MNIST", train=False, download=True, transform=transform)


In [ ]:
train_size = int(0.9 * len(mnist_data_train))       # Size of the train split
val_size = len(mnist_data_train) - train_size       # Size of the validation split


train_data, val_data = random_split(mnist_data_train, [train_size, val_size])

mask_set = [0, 1, 2, 3]
device_masks = torch.randint(0, 5, (len(mask_set), 196))

# Dictionary of all the rows of each mask-set in every file 
json_data = {
    "365nm":{
        "I1":range(0, 5),
        "I2":range(10, 15),
        "I3":range(18, 23),
        "I4":range(25, 30)
    },
    "455nm":{
        "I1":range(0, 5),
        "I2":range(7, 12),
        "I3":range(14, 19),
        "I4":range(21, 26)
    },
    "White":{
        "I1":range(0, 5),
        "I2":range(9, 14),
        "I3":range(16, 21),
        "I4":range(24, 29)
    }
}

In [ ]:
filename = "White"  # Excel document name from which data has to be extracted.
path = "data/"+filename+".xlsx" # Excel doc path

df = pd.read_excel(path, usecols='B:Q') # Read the excel sheet
# Break the sheet down into different `DataFrame`s for every Mask-set in the table
tables = [df.iloc[json_data[filename][key]].copy().reset_index(drop=True) for key in list(json_data[filename].keys())]
combined_table = pd.concat(tables, axis=0)

In [ ]:

# class CustomMNISTDataset(Dataset):
#     """Dataset object to save the preprocessed mnist dataset

#     Parameters -
#     mnist_data : `torch.utils.data.dataset`
#                 Contains images and corresponding labels of MNIST dataset               
#     tables : `list[pd.DataFrame]`
#                 conductance tables for every mask-set.
#     device_indices : `torch.tensor`
#                 Initial conductance states of every device. Shape of `((len(mask_sets), 196))`
#     mask_sets : `List[int]`
#                 List of all the mask-sets to be used

#     Attributes - 

#     mask_sets : `List[int]`
#                 A list of all the mask-sets used for simulation.
#     processed_data : `torch.tensor`
#                 MNIST images after preprocessing. Shape of ((N_samples, 1, 196 * len(mask_sets)))
#     labels : `torch.tensor`
#                 label of each corresponding image. Shape of ((N_samples, num_classes))   (num_classes = 10)
#     device_indices : List[List]
#                 Initial states of each device for every mask-set
#     """
#     def __init__(self, mnist_data, tables, device_indices, mask_sets ):
#         self.mask_sets = mask_sets
#         self.processed_data = []
#         self.labels = []

#         self.device_indices = device_indices.int().tolist()
        
#         # Preprocessing step. Same as discussed in the paper given.
#         for idx in tqdm(range(len(mnist_data))):
#             image, label = mnist_data[idx]
#             image = image.reshape(1, 196, 4)
#             image_combined = (
#                 image[:, :, 0] * 1000 + 
#                 image[:, :, 1] * 100 + 
#                 image[:, :, 2] * 10 + 
#                 image[:, :, 3]
#             )
#             # x = []
#             # for m_idx, mask in enumerate(self.mask_sets):
#             #     for i, device in enumerate(self.device_indices[m_idx]):                
#             #         x.append(tables[mask].loc[int(device), int(image_combined[0][i])]*1e9)
#             x = [
#                 tables[mask].loc[int(device), int(image_combined[0][i])] * 1e9
#                 for mask_idx, mask in enumerate(self.mask_sets)
#                 for i, device in enumerate(self.device_indices[mask_idx])
#             ]
            
#             self.processed_data.append(x)
#             self.labels.append(label)
        
#         # Convert the lists to `torch.tensor`
#         self.processed_data = torch.tensor(self.processed_data)
#         self.labels = torch.tensor(self.labels)

#     def __len__(self):
#         """ Function to access length of the dataset object.

#         Returns: `int`
#                     Length of the dataset object
#         """
#         return len(self.processed_data)

#     def __getitem__(self, idx):
#         """ Returns element's image and label at a given index.
#         Parameters - 
#         idx : int
#             index of the element to be accessed

#         Returns - torch.tensor, torch.tensor
#         """
#         return self.processed_data[idx], self.labels[idx]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, combined_table:pd.DataFrame, device_masks, mask_set):
        self.processed_data = []
        self.labels = []

        for idx in tqdm(range(len(data))):
            image, label = data[idx]
            image = image.reshape(196, 4)
            optical_pulses = (
                image[ :, 0] * 1000 + 
                image[ :, 1] * 100 + 
                image[ :, 2] * 10 + 
                image[ :, 3]
            ).repeat(len(mask_set))
            row_indices = torch.tensor([(device_masks[i]+mask_set[i]*5).tolist() for i in range(len(mask_set))]).flatten()
            column_indices = combined_table.columns.get_indexer(optical_pulses.tolist())
            
            self.processed_data.append(   combined_table.values[row_indices, column_indices] *1e9 )
            self.labels.append(label)
        self.processed_data = torch.tensor(np.array(self.processed_data)).to(device=device)
        self.labels = torch.tensor(self.labels).to(device=device)

    def __len__(self):
        return self.processed_data.shape[0]
    def __getitem__(self, index):
        return self.processed_data[index], self.labels[index]


In [ ]:

train_dataset = CustomDataset(train_data, combined_table, device_masks, mask_set)
validation_dataset = CustomDataset(val_data, combined_table, device_masks, mask_set)
test_dataset = CustomDataset(mnist_data_test, combined_table, device_masks, mask_set)

# train_dataset = CustomMNISTDataset(train_data, tables, device_masks, mask_set)
# validation_dataset = CustomMNISTDataset(val_data, tables, device_masks, mask_set)
# test_dataset = CustomMNISTDataset(mnist_data_test, tables, device_masks, mask_set)

100%|██████████| 10000/10000 [00:07<00:00, 1420.78it/s]


In [ ]:

# Model hyperparameters
BATCH_SIZE = 64
EPOCHS = 100
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class ReadoutLayer(nn.Module):
    """Readout layer.
    Parameters - 

    Attributes - 
    fc : `nn.Linear`
            Fully connected layer to be trained for reservoir computing.
    activation : `nn.functional.leaky_relu`
            Activation layer to be applied
    
    softmax : `nn.Softmax`
            Softmax activation function to get the one-hot encoded results.
    
    """
    def __init__(self, input_size):
        # super function to initialize the constructors of the parent classes.
        super(ReadoutLayer, self).__init__()
        # Class Attributes
        self.fc = nn.Linear(input_size, 10) 
        self.activation = nn.functional.leaky_relu
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        """Forward method to be executed on function call.
        Parameters - 
        x : torch.tensor
            Shape( Batch_size, 196 * len(mask_sets) )
        
        Returns : torch.tensor
            Shape( Batch_size, 10 )
        """
        x = self.fc(x)
        x = self.activation(x)
        x = self.softmax(x)
        return x

In [ ]:
model = ReadoutLayer(len(mask_set)*196).to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

val_accuracy, val_precision, val_recall, val_fscore = [], [], [], []

accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)

# Class-wise Confusion matrix
confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)



In [ ]:
for epoch in range(EPOCHS):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images.float())  # Forward pass
        loss = criterion(outputs, labels)  # Loss calculation
        optimizer.zero_grad()
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
    print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')
    
    # Validation phase
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images.float())
            preds = outputs.argmax(dim=1)

            # Update metrics
            accuracy.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)
            f1_score.update(preds, labels)

        # Print validation metrics
        print(f'Validation Accuracy: {accuracy.compute().item():.4f} Precision: {precision.compute().item():.4f} ', end="")
        print(f'Validation Recall: {recall.compute().item():.4f} F1 Score: {f1_score.compute().item():.4f}')

        # Updating the list to save current metrics
        val_accuracy.append(accuracy.compute().item())
        val_precision.append(precision.compute().item())
        val_recall.append(recall.compute().item())
        val_fscore.append(f1_score.compute().item())

        # Reset metrics for the next epoch
        accuracy.reset()
        precision.reset()
        recall.reset()
        f1_score.reset()
        confusion_matrix.reset()

Epoch [1/100], Loss: 1.7813
Validation Accuracy: 0.6600 Precision: 0.4784 Validation Recall: 0.6435 F1 Score: 0.5443
Epoch [2/100], Loss: 1.6320
Validation Accuracy: 0.8055 Precision: 0.7406 Validation Recall: 0.7974 F1 Score: 0.7635
Epoch [3/100], Loss: 1.6327
Validation Accuracy: 0.8142 Precision: 0.7527 Validation Recall: 0.8066 F1 Score: 0.7740
Epoch [4/100], Loss: 1.6118
Validation Accuracy: 0.8762 Precision: 0.8763 Validation Recall: 0.8738 F1 Score: 0.8733
Epoch [5/100], Loss: 1.5886
Validation Accuracy: 0.8980 Precision: 0.8968 Validation Recall: 0.8962 F1 Score: 0.8962
Epoch [6/100], Loss: 1.5443
Validation Accuracy: 0.8990 Precision: 0.8985 Validation Recall: 0.8986 F1 Score: 0.8975
Epoch [7/100], Loss: 1.5783
Validation Accuracy: 0.9013 Precision: 0.9006 Validation Recall: 0.8999 F1 Score: 0.8999
Epoch [8/100], Loss: 1.5727
Validation Accuracy: 0.9020 Precision: 0.9021 Validation Recall: 0.9007 F1 Score: 0.9005
Epoch [9/100], Loss: 1.5874
Validation Accuracy: 0.9040 Precisio

In [ ]:

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        outputs = model(torch.tensor(images, dtype=torch.float32))
        _, predicted = torch.max(outputs, 1)

        # Append predictions and labels for metric calculations
        all_preds.append(predicted)
        all_labels.append(labels)


# Concatenate all predictions and labels
all_preds = torch.cat(all_preds).to(device)
all_labels = torch.cat(all_labels).to(device)

# Calculate metrics
test_accuracy = accuracy(all_preds, all_labels)
test_precision = precision(all_preds, all_labels)
test_recall = recall(all_preds, all_labels)
test_f1 = f1_score(all_preds, all_labels)
test_confusion_matrix = confusion_matrix(all_preds, all_labels)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision*100:.4f}%')
print(f'Test Recall: {test_recall*100:.4f}%')
print(f'Test F1 Score: {test_f1:.4f}')
print("Confusion Matrix:")
print(test_confusion_matrix)

        

Test Accuracy: 92.05%
Test Precision: 92.0211%
Test Recall: 91.9254%
Test F1 Score: 0.9192
Confusion Matrix:
tensor([[ 962,    0,    2,    1,    1,    2,    7,    2,    3,    0],
        [   0, 1108,    5,    4,    1,    3,    4,    0,   10,    0],
        [  14,    5,  920,   17,   16,    1,   13,   14,   29,    3],
        [   5,    1,   21,  915,    4,   21,    1,    9,   23,   10],
        [   3,    2,    2,    1,  922,    2,   11,    3,    8,   28],
        [  10,    2,    0,   46,   16,  747,   19,    7,   39,    6],
        [  15,    2,    3,    2,    7,   11,  910,    1,    7,    0],
        [   1,   10,   24,    6,   14,    0,    0,  947,    2,   24],
        [  10,    8,    3,   22,    9,   14,    7,    8,  889,    4],
        [   9,    4,    0,   11,   48,    8,    1,   26,   17,  885]],
       device='cuda:0')


C:\Users\adith\AppData\Local\Temp\ipykernel_15348\247194469.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


In [ ]:
def get_metrics(filename, mask_set, device_masks):
    path = "data/"+filename+".xlsx" # Excel doc path

    df = pd.read_excel(path, usecols='B:Q') # Read the excel sheet
    # Break the sheet down into different `DataFrame`s for every Mask-set in the table
    tables = [df.iloc[json_data[filename][key]].copy().reset_index(drop=True) for key in list(json_data[filename].keys())]
    combined_table = pd.concat(tables, axis=0)

    train_dataset = CustomDataset(train_data, combined_table, device_masks, mask_set)
    validation_dataset = CustomDataset(val_data, combined_table, device_masks, mask_set)
    test_dataset = CustomDataset(mnist_data_test, combined_table, device_masks, mask_set)

    BATCH_SIZE = 64
    EPOCHS = 100
    learning_rate = 0.001
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    

    model = ReadoutLayer(len(mask_set)*196).to(device=device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)

    val_accuracy, val_precision, val_recall, val_fscore = [], [], [], []

    accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
    precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
    recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
    f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)

    # Class-wise Confusion matrix
    confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)

    for epoch in range(EPOCHS):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move to device
            outputs = model(images.float())  # Forward pass
            loss = criterion(outputs, labels)  # Loss calculation
            optimizer.zero_grad()
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images.float())
                preds = outputs.argmax(dim=1)

                # Update metrics
                accuracy.update(preds, labels)
                precision.update(preds, labels)
                recall.update(preds, labels)
                f1_score.update(preds, labels)

            # Print validation metrics
            print(f'Validation Accuracy: {accuracy.compute().item():.4f} Precision: {precision.compute().item():.4f} ', end="")
            print(f'Validation Recall: {recall.compute().item():.4f} F1 Score: {f1_score.compute().item():.4f}')

            # Updating the list to save current metrics
            val_accuracy.append(accuracy.compute().item())
            val_precision.append(precision.compute().item())
            val_recall.append(recall.compute().item())
            val_fscore.append(f1_score.compute().item())

            # Reset metrics for the next epoch
            accuracy.reset()
            precision.reset()
            recall.reset()
            f1_score.reset()
            confusion_matrix.reset()

    
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            # Move images and labels to GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(torch.tensor(images, dtype=torch.float32))
            _, predicted = torch.max(outputs, 1)

            # Append predictions and labels for metric calculations
            all_preds.append(predicted)
            all_labels.append(labels)


    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds).to(device)
    all_labels = torch.cat(all_labels).to(device)

    # Calculate metrics
    test_accuracy = accuracy(all_preds, all_labels)
    test_precision = precision(all_preds, all_labels)
    test_recall = recall(all_preds, all_labels)
    test_f1 = f1_score(all_preds, all_labels)
    test_confusion_matrix = confusion_matrix(all_preds, all_labels)

    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
    print(f'Test Precision: {test_precision*100:.4f}%')
    print(f'Test Recall: {test_recall*100:.4f}%')
    print(f'Test F1 Score: {test_f1:.4f}')
    print("Confusion Matrix:")
    print(test_confusion_matrix)

    save_path = "data/mnist_results_debug/" + filename + '_' + ''.join(map(str, mask_set)) + '.npz'
    np.savez(save_path,
            predictions = all_preds,
            labels = all_labels,
            validation_accuracy = val_accuracy,
            validation_precision = val_precision,
            validation_recall = val_recall,
            validation_fscore = val_fscore
            
    )
    return test_accuracy, test_precision, test_recall, test_f1

        



        

In [ ]:

def get_all_combinations(lis):
    all_combinations = []
    for r in range(1, len(lis)+1):
        combs = list(itertools.combinations(lis, r))
        all_combinations.extend(combs)
    return [list(ele) for ele in all_combinations]

In [ ]:
metrics_json = {
    "White":{},
    "365nm":{},
    "455nm":{}
}
mask_sets = [0]
combinations = get_all_combinations(mask_sets)
for filename in ["White", "455nm"]:
    for ms in combinations:

        device_masks = torch.randint(0, 5, (len(ms), 196))

        accuracy, precision, recall, fscore = get_metrics(filename=filename, mask_set=ms, device_masks=device_masks)
        metrics_json[filename][''.join(map(str, ms)) ] = {
            "Accuracy":accuracy.item(),
            "Trainable Parameters": (len(ms)*196+1)*10,
            "Precision":precision.item(),
            "Recall":recall.item(),
            "F-score":fscore.item()
            
        }

            

100%|██████████| 10000/10000 [00:04<00:00, 2013.39it/s]


Epoch [1/100], Loss: 2.0553
Validation Accuracy: 0.3862 Precision: 0.1608 Validation Recall: 0.3792 F1 Score: 0.2237
Epoch [2/100], Loss: 2.0924
Validation Accuracy: 0.4350 Precision: 0.2286 Validation Recall: 0.4375 F1 Score: 0.2947
Epoch [3/100], Loss: 1.8848
Validation Accuracy: 0.6095 Precision: 0.4344 Validation Recall: 0.6047 F1 Score: 0.5015
Epoch [4/100], Loss: 1.9152
Validation Accuracy: 0.6162 Precision: 0.4385 Validation Recall: 0.6115 F1 Score: 0.5065
Epoch [5/100], Loss: 1.9268
Validation Accuracy: 0.6208 Precision: 0.4413 Validation Recall: 0.6163 F1 Score: 0.5106
Epoch [6/100], Loss: 1.8230
Validation Accuracy: 0.6240 Precision: 0.4443 Validation Recall: 0.6199 F1 Score: 0.5139
Epoch [7/100], Loss: 1.9379
Validation Accuracy: 0.6885 Precision: 0.5658 Validation Recall: 0.6852 F1 Score: 0.6131
Epoch [8/100], Loss: 1.7411
Validation Accuracy: 0.6893 Precision: 0.5627 Validation Recall: 0.6865 F1 Score: 0.6125
Epoch [9/100], Loss: 1.7953
Validation Accuracy: 0.6913 Precisio

C:\Users\adith\AppData\Local\Temp\ipykernel_24480\2894068989.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


Test Accuracy: 81.04%
Test Precision: 74.2312%
Test Recall: 80.9573%
Test F1 Score: 0.7702
Confusion Matrix:
tensor([[ 952,    0,    2,    2,    1,    5,    9,    4,    5,    0],
        [   0, 1100,    5,    5,    0,    3,    4,    1,   17,    0],
        [  18,   16,  869,   17,   21,    2,   24,   15,   50,    0],
        [  13,    1,   18,  875,    2,   35,    4,   25,   37,    0],
        [   4,    6,    3,    1,  931,    2,   12,   11,   12,    0],
        [  15,   10,    4,   51,   25,  713,   17,   19,   38,    0],
        [  17,    3,    9,    0,   12,   19,  890,    3,    5,    0],
        [   4,   21,   24,   10,   15,    2,    1,  946,    5,    0],
        [  17,   12,   11,   23,   21,   27,   15,   20,  828,    0],
        [  15,    7,    8,   16,  445,   43,    2,  354,  119,    0]])


100%|██████████| 10000/10000 [00:05<00:00, 1985.87it/s]


Epoch [1/100], Loss: 1.9592
Validation Accuracy: 0.5742 Precision: 0.4125 Validation Recall: 0.5882 F1 Score: 0.4821
Epoch [2/100], Loss: 1.9019
Validation Accuracy: 0.5965 Precision: 0.4290 Validation Recall: 0.6091 F1 Score: 0.5002
Epoch [3/100], Loss: 1.7454
Validation Accuracy: 0.6802 Precision: 0.5684 Validation Recall: 0.6896 F1 Score: 0.6163
Epoch [4/100], Loss: 1.6782
Validation Accuracy: 0.6853 Precision: 0.5705 Validation Recall: 0.6939 F1 Score: 0.6204
Epoch [5/100], Loss: 1.8482
Validation Accuracy: 0.6862 Precision: 0.5699 Validation Recall: 0.6949 F1 Score: 0.6208
Epoch [6/100], Loss: 1.8743
Validation Accuracy: 0.6903 Precision: 0.5759 Validation Recall: 0.6990 F1 Score: 0.6251
Epoch [7/100], Loss: 1.8121
Validation Accuracy: 0.6905 Precision: 0.5719 Validation Recall: 0.6992 F1 Score: 0.6245
Epoch [8/100], Loss: 1.8095
Validation Accuracy: 0.6912 Precision: 0.5725 Validation Recall: 0.6996 F1 Score: 0.6249
Epoch [9/100], Loss: 1.7725
Validation Accuracy: 0.6923 Precisio

C:\Users\adith\AppData\Local\Temp\ipykernel_24480\2894068989.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


In [ ]:

filename = 'data/metrics_v2.xlsx'

with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    for wavelength, results in metrics_json.items():
        # Flatten the data for each wavelength
        flattened_data = []
        for index, metrics in results.items():
            if metrics:  # Check if the metrics are not empty
                flattened_data.append({
                    'Index': index,
                    'Accuracy': metrics['Accuracy'],  # Convert tensor to float
                    'Precision': metrics['Precision'],
                    'Recall': metrics['Recall'],
                    'F-score': metrics['F-score']
                })
        
        # Create a DataFrame for the current wavelength
        if flattened_data:  # Check if there's data to save
            df = pd.DataFrame(flattened_data)
            # Write the DataFrame to a specific sheet named after the wavelength
            df.to_excel(writer, sheet_name=wavelength, index=False)

print("Data saved to", filename)

Data saved to data/metrics_v2.xlsx
